### Saving The Uploaded Filed


1. Create a helper function inside your views.py (not in the IndexView class!) and call it store_file.
2. This function should have an input argument called file.
3. Inside this function use python to write the file in a directory called 'temp/' and add a name for the name you want for the file.
    - Please undertand that for now you need to choose the extension of you file exactly alike your file which you are going to upload. Later I will tell you how to do it automatically.
4. Set the mode to 'wb+'
5. As you know from our python course, you can now just call the write command for saving the file, but it is not a good approach, let me explain why:
    - When you write the whole data in a file, the system will read it entirely and move it to the ram! What if you dont have anough space in your ram? or what if a great number of users decide to upload a file at the same time to your server? You see where I am going?
    - so instead of writing the whole file as once we cut it into chunks and then write it chunk by chunk untill it is finished.
    - to do so, write a for loop that runs for every chunk in file.chunks(), and inside the for loop we write it to the file.
6. Now, instead of printing the file, call the store_file function and pass it the file.
7. Don't forget to add the 'temp' folder inside your project directory otherwise you will get a 'FileNotFoundError'.
    - Please understand that the the python code we wrote in the store_file function will assume the root of your project as the root place for loking for the 'temp' dicrectory and not the app directory.
    - To make sure that you won't make a mistake: add the temp folder next to your manage.py file.

Your views.py file should look like the following block:

In [ ]:
from django.shortcuts import render
from django.views import View
from django.http import HttpResponseRedirect

def store_file(file):
    with open('temp/image.jpg', 'wb+') as destination:
        for chunk in file.chunks():
            destination.write(chunk)


class IndexView(View):
    def get(self, request):
        return render(request, 'profiles/index.html')
    
    def post(self, request):
        store_file(request.FILES['image'])
        return HttpResponseRedirect('/profiles/')

What we have done here, is not a good aproach for file upload and saving! You may ask why?<br>
Well, this approach is not flexible (only works with jpg files) and requires alot of coding.<br>
Also, It is not safe! (I will explain this later).<br>

### Simplifying File Uploads:

What you need to understand is that file uploads are just another sort of forms! and in the case of forms, django has provided us with a better form of handling forms which is using forms.py<br>

1. Create a forms.py file inside your app directory.
2. from django import forms
3. Create a form class called ProfileForm and inherit from forms/Form
4. Define a class attribute with the value of forms.FileField() and call it user_image.
5. You can configure this by passing a number of arguments to the FileField method, for example:
    - allow_empty_file [True/False]
    - ...
6. We leave the FileField method without any arguments at the moment.
7. Navigate to your views.py and from .forms import ProfileForm
8. Create an instance of it in the get method
8. Pass it in the render function inside the get method under the key name of 'form'

You forms.py file should look like the following block:

In [ ]:
from django import forms 

class ProfileForm(forms.Form):
    user_image = forms.FileField()

Your views.py file should look like the following block:

In [ ]:
from django.shortcuts import render
from django.views import View
from django.http import HttpResponseRedirect
from .forms import ProfileForm

def store_file(file):
    with open('temp/image.jpg', 'wb+') as destination:
        for chunk in file.chunks():
            destination.write(chunk)


class IndexView(View):
    def get(self, request):
        form = ProfileForm()
        return render(request, 'profiles/index.html', {
            'form':form
        })
    
    def post(self, request):
        store_file(request.FILES['image'])
        return HttpResponseRedirect('/profiles/')


10. Now, inside your template you can replace the input tag with {{ form }}<br>


You could show it indetails too!<br>

You index.html template should look like the following block:

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block page_title %}Create a Profile{% endblock page_title %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "profiles/index.css" %}">
{% endblock css_files %}

{% block content %}
    <div class='main-wrapper'>
        <form action="/profiles/" method='POST' enctype='multipart/form-data'>
            {% csrf_token %}
            {{ form }}
            <button>Upload</button>
        </form>
    </div>
{% endblock content %}

What we have done sofar doesn't handle the file upload process and we still need to do something about the post method:

The least we can do here is to use 'ProfileForm(request.POST, request.FILES)' and store it in a variable. This way we can validate both the input fields (Yes, we don't have any at the moment but we could have...) and input files using is_valid().

After validating the inputs, we call the store_file method and redirect to '/profiles' and if the form is not valid we can get back to the same page, and show the errors.

Your views.py file should look like the following block:

In [ ]:
from django.shortcuts import render
from django.views import View
from django.http import HttpResponseRedirect
from .forms import ProfileForm

def store_file(file):
    with open('temp/image.jpg', 'wb+') as destination:
        for chunk in file.chunks():
            destination.write(chunk)


class IndexView(View):
    def get(self, request):
        form = ProfileForm()
        return render(request, 'profiles/index.html', {
            'form':form
        })
    
    def post(self, request):
        submited_form = ProfileForm(request.POST, request.FILES)
        if submited_form.is_valid():
            store_file(request.FILES['image'])
            return HttpResponseRedirect('/profiles/')
        else:
            return render(request, 'profiles/index.html', {
                "form":submited_form
            })

What we have done in the post method will make sure that the user doesn't bypass the required inputs and other validations that might be defined in the form.<br>

We haven't yet simplified the storing of the file. To do that we need to combine the form with a model.

You will learn about it in the next notebook<br>

Before you go to the next notebook, make some changes in the css file so the labels are being shown better.<br>

My index.css file looks like the following block:

In [ ]:
.main-wrapper{
    width:30rem;
    max-width: 30rem;
    color:white;
    border:1px solid rgb(93, 83, 83);
    padding:3rem;
    margin:5rem auto 0 auto;
    border-radius: 8px;
}
.main-wrapper input,
.main-wrapper button{
    display:block;
}
.main-wrapper input[type='file']{
    width:90%;
    margin:auto;
    font-size: 1.2rem;
}
.main-wrapper button{
    margin:2rem auto 0 auto;
    width:90%;
    height:2.5rem;
    font-size: 1.3rem;
    background-color:rgb(61, 130, 235);
    color:white;
    border:1px solid rgb(61, 130, 235);
    border-radius: 8px;
}
.main-wrapper button:hover,
.main-wrapper button:focus{
    background-color:rgb(26, 82, 165);
    border-color:rgb(26, 82, 165);
}
.main-wrapper label{
    display:block;
    width:90%;
    margin:auto;
    padding:0.5rem 0 0.5rem 0;
}
.main-wrapper ul{
    width:90%;
    display:block;
}
.main-wrapper ul li{
    width:90%;
    margin:auto;
    color:rgb(163, 27, 27);
}